NOTE: this should eventually be converted to a Python script once the algorithm to determin the sign point is finalised

In [31]:
import pandas as pd
import numpy as np
pd.set_option("display.notebook_repr_html", False)

In [32]:
df = pd.read_csv("../data/processed/170329/predictions/df_garmin_joined.csv")
df.shape

(76, 20)

In [33]:
# initialise cluster related columns
df["cluster_id"] = -1 # means that this row has not been id'ed as a sign by model
df["is_cluster_main"] = 0 # means that this is the closest point to the centre
df["num_in_cluster"] = -1 # only for rows where is_cluster_main = 1

In [34]:
def get_main_cluster_point(cluster_members, df):
    #find member closest to middle
    middle = 1920/2 #middle pixel in x direction
    closest_id = cluster_members[0]
    closest_distance =  np.abs( df.loc[ df["id"]==closest_id, "x"].values[0] - middle )
    for id in cluster_members[1:]:
        d =  np.abs( df.loc[ df["id"]==id, "x" ].values[0] - middle)
        #print "distance = %d" % d
        if d < closest_distance:
            closest_id = id
            closest_distance = d
    #print "closest distance = %d" % closest_distance
    return closest_id

In [38]:
def lookahead(iterable):
    """Pass through all values from the given iterable, augmented by the
    information if there are more values to come after the current one
    (True), or if it is the last value (False).
    """
    # Get an iterator and pull the first value.
    it = iter(iterable)
    last = next(it)
    # Run the iterator to exhaustion (starting from the second value).
    for val in it:
        # Report the *previous* value (more to come).
        yield last, True
        last = val
    # Report the last value.
    yield last, False

In [47]:
import pprint
cluster_id = 1
am_mid_cluster = False
frames_between_clusters = 24 #threshold of empty frames before I start a new cluster
n_to_process = 1e6 #set to large number to process all frames
n_processed = 0
#base_frame_i = 18261
#base_frame_f = 19000
#for _, row in df[ df["base_frame"].between(base_frame_i, base_frame_f) ].iterrows():
for ((_, row), has_more) in lookahead(df.iterrows()):
    #print "base_frame = {}".format(row["base_frame"])
    base_frame = row["base_frame"]
    is_sign = row["accum"] > 0.4 and row["score"] > 0
    #is_sign = row["ymain"] == 1
    #end the cluster if no sign seen for frame_between_clusters frames 
    #or sign in frame but this is last frame
    end_cluster =  (~is_sign and (base_frame-latest_cluster_frame > frames_between_clusters))\
                   or (is_sign and not has_more)
    if not am_mid_cluster and is_sign:
        #start of new cluster
        print "new cluster starting at frame %d" % row["base_frame"]
        latest_cluster_frame = base_frame
        am_mid_cluster = True
        cluster_members = [row["id"]]
    elif not am_mid_cluster and not is_sign:
        #not in a cluster and no sign in frame. Keep moving
        pass
    elif am_mid_cluster:
        if is_sign: 
            print "found sign"
            #sign in frame. Add to cluster members 
            cluster_members.append(row["id"])
            latest_cluster_frame = base_frame
        if end_cluster:
            # This is the end of the cluster
            print "end of cluster. cluster_members"
            pprint.pprint(cluster_members)
            df.loc[ df["id"].isin(cluster_members), "cluster_id" ] = cluster_id
            cluster_id += 1
            cluster_point_id = get_main_cluster_point(cluster_members, df)
            df.loc[ df["id"]==cluster_point_id, "is_cluster_main"] = 1
            df.loc[ df["id"]==cluster_point_id, "num_in_cluster"] = len(cluster_members)
            #reset cluster variables
            am_mid_cluster = False
    else:
        print "WTF"
        print am_mid_cluster
        print is_sign
    n_processed += 1
    if n_processed % 1000 == 0:
        print n_processed
    if n_processed > n_to_process:
        break

new cluster starting at frame 19248
found sign
found sign
found sign
found sign
found sign
found sign
end of cluster. cluster_members
['f0019248_x0758_y0378',
 'f0019252_x0966_y0384',
 'f0019260_x1416_y0403',
 'f0019284_x1181_y0526',
 'f0019288_x1415_y0541',
 'f0019292_x1655_y0547',
 'f0019296_x1885_y0561']
new cluster starting at frame 19328
found sign
end of cluster. cluster_members
['f0019328_x0596_y0387', 'f0019332_x0962_y0404']
new cluster starting at frame 19504
found sign
end of cluster. cluster_members
['f0019504_x0255_y0364', 'f0019508_x0650_y0359']
new cluster starting at frame 19672
found sign
found sign
found sign
found sign
end of cluster. cluster_members
['f0019672_x0145_y0171',
 'f0019676_x0315_y0186',
 'f0019680_x0537_y0182',
 'f0019684_x0742_y0204',
 'f0019688_x0933_y0215']


In [48]:
df["cluster_id"].unique()

array([-1,  1,  2,  3,  4])

In [49]:
df.columns

Index([u'Unnamed: 0', u'id', u'Unnamed: 0.1', u'accum', u'area', u'base_frame',
       u'blue_pix', u'frac_blue_area', u'frac_r_y', u'frac_red_area',
       u'frame_num', u'lat', u'lon', u'radius', u'red_pix', u'x', u'ymain',
       u'score', u'ypred', u'image_path', u'cluster_id', u'is_cluster_main',
       u'num_in_cluster'],
      dtype='object')

In [50]:
df.head()

   Unnamed: 0                    id  Unnamed: 0.1     accum         area  \
0           0  f0019204_x0087_y0609             0  0.250000  1963.495408   
1           1  f0019212_x0479_y0629             1  0.201389  2642.079422   
2           2  f0019216_x0892_y0874             2  0.215278  1963.495408   
3           3  f0019216_x0684_y0661             3  0.312500  1963.495408   
4           4  f0019220_x0899_y0675             4  0.326389  1963.495408   

   base_frame  blue_pix  frac_blue_area  frac_r_y  frac_red_area  \
0       19204       523        0.266362         0       0.256176   
1       19212       718        0.271756         0       0.235421   
2       19216       506        0.257704         0       0.237332   
3       19216       451        0.229692         0       0.283678   
4       19220       373        0.189967         0       0.293354   

       ...        radius  red_pix    x  ymain     score  ypred  \
0      ...            25      503   87    609 -0.188783      0   
1 

In [51]:
df["num_in_cluster"].unique()

array([-1,  7,  2,  5])

In [52]:
df.to_csv("../data/processed/170329/predictions/df_garmin_joined_clustered.csv")